In [1]:
import sys
import torch
from os.path import join
import pandas as pd
import numpy as np 

# Paths
DATA_PATH = '/projects/b1042/GoyalLab/jaekj/merlin_cxg_2023_05_15_sf-log1p'

cell_type_mapping = pd.read_parquet(join(DATA_PATH, 'categorical_lookup/cell_type.parquet'))
tissue_general_mapping = pd.read_parquet(join(DATA_PATH, 'categorical_lookup/tissue_general.parquet'))
cell_type_hierarchy = np.load(join(DATA_PATH, 'cell_type_hierarchy/child_matrix.npy'))

Train embedding

In [2]:
# Load the embedding if not already in memory
train_emb = torch.load("train_embedding.pt")
X_train = train_emb["X"]  # shape: n_cells x embedding_dim
y_train = train_emb["y_true"]  # cell type labels as integers

# Convert to a DataFrame for easier grouping
df_train = pd.DataFrame(X_train.numpy())
df_train['cell_type'] = y_train.numpy()

# Aggregate by cell type (e.g., mean embedding)
celltype_emb_train = df_train.groupby('cell_type').mean()

print("Cell type embedding shape:", celltype_emb_train.shape)
cell_type_mapping_lookup = cell_type_mapping['label'].to_dict() 
celltype_emb_train['cell_type_name'] = celltype_emb_train.index.map(cell_type_mapping_lookup)
celltype_emb_train.index = celltype_emb_train['cell_type_name']
celltype_emb_train = celltype_emb_train.drop(columns='cell_type_name')

print(celltype_emb_train.head())

# Save
torch.save(celltype_emb_train, "train_celltype_embedding.pt")

/tmp/ipykernel_2147080/3908554019.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_emb = torch.load("train_embedding.pt")


Cell type embedding shape: (164, 128)
                                                      0         1         2    \
cell_type_name                                                                  
B cell                                           0.216572  1.657566  0.486447   
Bergmann glial cell                              0.034370  0.323564  0.012756   
CD14-low, CD16-positive monocyte                 1.254785  0.106149  1.224594   
CD14-positive monocyte                           0.283573  0.431686  0.590523   
CD14-positive, CD16-negative classical monocyte  0.419099  0.008193  0.391868   

                                                      3         4         5    \
cell_type_name                                                                  
B cell                                           1.440559  0.300549  1.048494   
Bergmann glial cell                              0.152558  0.598884  0.645573   
CD14-low, CD16-positive monocyte                 0.055330  0.161077  0

In [3]:
train_celltype_embedding = torch.load("train_celltype_embedding.pt")
train_celltype_embedding

/tmp/ipykernel_2147080/1221254664.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_celltype_embedding = torch.load("train_celltype_embedding.pt")


,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
cell_type_name,,,,,,,,,,,,,,,,,,,,,
B cell,0.216572,1.657566,0.486447,1.440559,0.300549,1.048494,0.277483,0.999601,1.216470,2.274750,...,0.493398,0.342398,0.238439,0.979816,0.438486,1.491465,0.771414,0.726889,0.308585,0.138412
Bergmann glial cell,0.034370,0.323564,0.012756,0.152558,0.598884,0.645573,0.113513,5.043242,0.042322,0.095867,...,0.003905,2.624892,0.664788,4.047603,0.669451,3.283303,1.130970,4.268102,0.047337,4.624674
"CD14-low, CD16-positive monocyte",1.254785,0.106149,1.224594,0.055330,0.161077,0.066573,0.133279,0.803708,0.143405,3.261189,...,0.819440,0.568955,0.001435,0.012891,1.093038,1.980633,0.207958,0.319596,0.588092,0.003471
CD14-positive monocyte,0.283573,0.431686,0.590523,0.683262,4.097441,0.568314,0.228310,0.052372,1.455893,1.465279,...,0.547172,0.288900,0.175101,0.074864,0.187308,0.119150,1.203330,0.391701,1.903133,0.038819
"CD14-positive, CD16-negative classical monocyte",0.419099,0.008193,0.391868,0.707985,2.450489,1.282106,0.267402,1.309367,2.174791,1.101662,...,0.547562,0.251523,0.260538,0.463681,1.479947,0.058100,1.701265,0.036234,0.007331,1.556411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
type I pneumocyte,1.963323,1.687371,0.150596,1.801764,2.248454,1.540785,0.183485,0.200096,0.014606,0.058918,...,0.022808,4.904091,4.836779,0.638974,0.458911,1.359379,0.208806,3.960306,0.105963,0.718800
type II pneumocyte,1.633867,0.096628,0.807107,0.333062,1.547208,0.328199,1.711047,0.019672,0.739604,0.119503,...,0.119684,3.510922,1.853047,0.367132,0.123250,1.878841,1.362033,1.889813,3.343423,0.313333
vascular associated smooth muscle cell,2.400663,0.116486,0.022035,0.438552,0.246289,0.934753,0.019481,0.641448,1.069372,1.501310,...,1.779522,4.342218,0.035066,1.753723,0.303655,1.091368,0.584966,0.068827,0.022718,0.811470


Test embedding

In [4]:
# Load the embedding if not already in memory
test_emb = torch.load("test_embedding.pt")
X_test = test_emb["X"]  # shape: n_cells x embedding_dim
y_test = test_emb["y_true"]  # cell type labels as integers

# Convert to a DataFrame for easier grouping
df_test = pd.DataFrame(X_test.numpy())
df_test['cell_type'] = y_test.numpy()

# Aggregate by cell type (e.g., mean embedding)
celltype_emb_test = df_test.groupby('cell_type').mean()

print("Cell type embedding shape:", celltype_emb_test.shape)

cell_type_mapping_lookup = cell_type_mapping['label'].to_dict()  # int -> name
celltype_emb_test['cell_type_name'] = celltype_emb_test.index.map(cell_type_mapping_lookup)
celltype_emb_test.index = celltype_emb_test['cell_type_name']
celltype_emb_test = celltype_emb_test.drop(columns='cell_type_name')

print(celltype_emb_test.head())

# Save
torch.save(celltype_emb_test, "test_celltype_embedding.pt")

/tmp/ipykernel_2147080/3642434945.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test_emb = torch.load("test_embedding.pt")


Cell type embedding shape: (164, 128)
                                                      0         1         2    \
cell_type_name                                                                  
B cell                                           0.257409  1.658625  0.456759   
Bergmann glial cell                              0.171717  0.478282  0.151182   
CD14-low, CD16-positive monocyte                 1.315203  0.128882  1.285513   
CD14-positive monocyte                           0.318037  0.504748  0.611588   
CD14-positive, CD16-negative classical monocyte  0.376883  0.010550  0.429454   

                                                      3         4         5    \
cell_type_name                                                                  
B cell                                           1.505509  0.287644  1.196042   
Bergmann glial cell                              0.486801  0.746065  0.720750   
CD14-low, CD16-positive monocyte                 0.069167  0.151732  0

In [5]:
test_celltype_embedding = torch.load("test_celltype_embedding.pt")
test_celltype_embedding

/tmp/ipykernel_2147080/1151867312.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test_celltype_embedding = torch.load("test_celltype_embedding.pt")


,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
cell_type_name,,,,,,,,,,,,,,,,,,,,,
B cell,0.257409,1.658625,0.456759,1.505509,0.287644,1.196042,0.357218,1.147294,1.236483,2.279870,...,0.454606,0.391975,0.247102,1.121474,0.463649,1.606995,1.004147,0.766818,0.390318,0.168435
Bergmann glial cell,0.171717,0.478282,0.151182,0.486801,0.746065,0.720750,0.194807,4.802727,0.069784,0.151938,...,0.007847,2.785421,0.756820,3.570024,0.886015,3.260195,1.267442,3.913872,0.051319,4.298329
"CD14-low, CD16-positive monocyte",1.315203,0.128882,1.285513,0.069167,0.151732,0.042235,0.118228,0.776777,0.172080,3.060999,...,0.741660,0.619189,0.004754,0.008498,1.244954,2.116615,0.201429,0.282141,0.640210,0.002253
CD14-positive monocyte,0.318037,0.504748,0.611588,0.708678,4.035848,0.601852,0.263356,0.051729,1.641764,1.376366,...,0.432322,0.319500,0.264856,0.059342,0.204161,0.152082,1.051117,0.349836,2.067016,0.046429
"CD14-positive, CD16-negative classical monocyte",0.376883,0.010550,0.429454,0.690054,2.423855,1.389023,0.300471,1.182426,1.949010,1.168738,...,0.490771,0.267432,0.276360,0.478688,1.441517,0.086366,1.763607,0.046468,0.011595,1.514239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
type I pneumocyte,1.732641,1.495965,0.260661,1.665460,2.237536,1.236339,0.299587,0.264417,0.052803,0.108943,...,0.032872,4.778317,4.276694,0.585070,0.496168,1.451959,0.293728,3.678295,0.181312,0.639997
type II pneumocyte,1.838210,0.103151,0.845236,0.438460,1.606852,0.335751,1.594931,0.045604,0.727514,0.140037,...,0.252053,3.881410,1.988914,0.334697,0.216667,1.688334,1.335991,2.075839,2.806583,0.291634
vascular associated smooth muscle cell,2.244081,0.434625,0.141924,0.673730,0.318450,1.037876,0.020437,1.055328,0.996146,1.843889,...,1.596617,4.579144,0.397187,1.770530,0.433029,1.000217,0.698922,0.168280,0.024226,0.731066


Validation Embedding

In [7]:
# Load the embedding if not already in memory
val_emb = torch.load("val_embedding.pt")
X_val = val_emb["X"]  # shape: n_cells x embedding_dim
y_val = val_emb["y_true"]  # cell type labels as integers

# Convert to a DataFrame for easier grouping
df_val = pd.DataFrame(X_val.numpy())
df_val['cell_type'] = y_val.numpy()

# Aggregate by cell type (e.g., mean embedding)
celltype_emb_val = df_val.groupby('cell_type').mean()

print("Cell type embedding shape:", celltype_emb_val.shape)

cell_type_mapping_lookup = cell_type_mapping['label'].to_dict()  # int -> name
celltype_emb_val['cell_type_name'] = celltype_emb_val.index.map(cell_type_mapping_lookup)
celltype_emb_val.index = celltype_emb_val['cell_type_name']
celltype_emb_val = celltype_emb_val.drop(columns='cell_type_name')

print(celltype_emb_val.head())

# Save
torch.save(celltype_emb_val, "val_celltype_embedding.pt")

/tmp/ipykernel_2147080/3216110610.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  val_emb = torch.load("val_embedding.pt")


Cell type embedding shape: (164, 128)
                                                      0         1         2    \
cell_type_name                                                                  
B cell                                           0.204062  1.714244  0.448190   
Bergmann glial cell                              0.087169  0.420670  0.033134   
CD14-low, CD16-positive monocyte                 1.503392  0.129430  1.163151   
CD14-positive monocyte                           0.333188  0.539114  0.565669   
CD14-positive, CD16-negative classical monocyte  0.363895  0.010146  0.412305   

                                                      3         4         5    \
cell_type_name                                                                  
B cell                                           1.351259  0.385545  0.887748   
Bergmann glial cell                              0.160788  0.631331  0.744243   
CD14-low, CD16-positive monocyte                 0.065418  0.185184  0

In [8]:
val_celltype_embedding = torch.load("val_celltype_embedding.pt")
val_celltype_embedding

/tmp/ipykernel_2147080/1808905215.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  val_celltype_embedding = torch.load("val_celltype_embedding.pt")


,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
cell_type_name,,,,,,,,,,,,,,,,,,,,,
B cell,0.204062,1.714244,0.448190,1.351259,0.385545,0.887748,0.264415,0.937795,1.279000,2.262093,...,0.656854,0.416434,0.179891,0.885045,0.427120,1.556212,0.703676,0.769598,0.265944,0.174219
Bergmann glial cell,0.087169,0.420670,0.033134,0.160788,0.631331,0.744243,0.167929,4.880132,0.072509,0.127608,...,0.000850,2.758264,0.744767,3.854766,0.709338,3.162307,1.193338,4.172861,0.061458,4.406897
"CD14-low, CD16-positive monocyte",1.503392,0.129430,1.163151,0.065418,0.185184,0.082240,0.133665,0.757332,0.168136,3.106228,...,0.879078,0.632837,0.002931,0.019808,1.048388,1.880138,0.222013,0.361028,0.587690,0.009349
CD14-positive monocyte,0.333188,0.539114,0.565669,0.635232,4.026418,0.635872,0.258941,0.047854,1.413710,1.367863,...,0.556612,0.296901,0.172794,0.098646,0.165941,0.139712,1.190743,0.395139,1.889531,0.055200
"CD14-positive, CD16-negative classical monocyte",0.363895,0.010146,0.412305,0.674149,2.517896,1.322456,0.303034,1.132469,2.017209,1.080455,...,0.510581,0.263200,0.265157,0.466029,1.414873,0.079297,1.654000,0.048815,0.013650,1.520057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
type I pneumocyte,2.136868,1.254519,0.316083,1.307839,2.348284,1.155442,0.321397,0.179543,0.045797,0.085223,...,0.028435,4.538895,4.428952,0.504798,0.230319,1.236435,0.330081,3.762540,0.168494,0.753103
type II pneumocyte,1.866745,0.108199,0.870476,0.277607,1.634224,0.253423,1.953153,0.035524,0.702097,0.160603,...,0.240852,3.475893,1.616047,0.353078,0.112449,1.459903,1.564091,2.032715,2.917799,0.428295
vascular associated smooth muscle cell,2.214467,0.352451,0.066712,0.808933,0.341766,1.185942,0.026617,0.613224,1.315241,1.564512,...,1.692693,5.011948,0.269056,1.657803,0.446089,1.082270,0.853700,0.173351,0.034570,0.859666
